In [3]:
import pandas as pd
import geopandas as gpd
import minimod
import matplotlib.pyplot as plt
from ipywidgets import fixed, interact, IntSlider

data_folder = 'Optimization Work/demewoz_lives_saved/data_files/'

effective_coverage_folder = data_folder + 'effective_coverage/'
cost_folder = data_folder + 'new_cost_benefits/'

In [7]:
zinc_eff_cov

,Zinc interventions,zoneName,inadequate_percent,inadequate_percent_SE,effective_coverage,effective_coverage_SE
0,Cube(600mg/kg)_EAREFSA,Cities,26.5,3.050176,14.6,3.778857
1,Cube(600mg/kg)_EAREFSA,North,21.1,2.461696,15.9,2.585362
2,Cube(600mg/kg)_EAREFSA,South,23.4,1.793554,22.3,3.803026
3,Cube(600mg/kg) and wheat flour(95mg/kg)_EAREFSA,Cities,9.5,1.217957,31.6,4.118308
4,Cube(600mg/kg) and wheat flour(95mg/kg)_EAREFSA,North,17.2,2.908833,19.8,3.208394
5,Cube(600mg/kg) and wheat flour(95mg/kg)_EAREFSA,South,14.4,3.086962,31.3,4.866500
6,Zinc baseline_EAREFSA,Cities,41.1,4.172411,0.0,0.000000
7,Zinc baseline_EAREFSA,North,37.0,2.687999,0.0,0.000000
8,Zinc baseline_EAREFSA,South,45.7,3.976921,0.0,0.000000
9,Wheat flour_EAREFSA,Cities,9.9,1.541104,31.2,3.935924


In [6]:
# Get intervention names to change
zinc_eff_cov = (
pd.read_excel(effective_coverage_folder + 'April_20_zinc_effective coverage.xlsx')
)

long_intervention_names = zinc_eff_cov['Vitamin A intervention'].unique().tolist()
short_intervention_names = ['cube',
'oilclinic',
'maxoil',
'maxoilcube',
'oil',
'maxoilcubeclinic',
'oilcube',
'oilcubeclinic',
'maxoilclinic',
'clinic',
'cubeclinic',
'vas',
'maxoilvas',
'maxoilvas',
'oilvas',
'oilvas',
'cubevas',
'oilcubevas',
'oilcubevas',
'maxoilcubevas'
]
intervention_rename_list = {k:v for k,v in zip(long_intervention_names, short_intervention_names)}

KeyError: 'Vitamin A intervention'